In [1]:
import napari
import numpy as np
from tifffile import imread
import tifffile
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.path as mplPath
from pathlib import Path
from matplotlib import pyplot as plt

### Load the crop you want to annotate 

In [3]:
### load the different staining 
image_name = "0_0_Cellbound1_4_crop4000_14_12.tif"
path_dapi_folder = Path("/home/tom/share/open_vizgen/HumanBreastCancerPatient1/crop4000v2/datasets_24_04_22_Cellbound1_4_nuclei_target/dapi/")
path_cyto1_folder = Path("/home/tom/share/open_vizgen/HumanBreastCancerPatient1/crop4000v2/datasets_24_04_22_Cellbound1_4_nuclei_target/cyto1/")
path_cyto2_folder = Path("/home/tom/share/open_vizgen/HumanBreastCancerPatient1/crop4000v2/datasets_24_04_22_Cellbound1_4_nuclei_target/cyto2/")
path_cyto3_folder = Path("/home/tom/share/open_vizgen/HumanBreastCancerPatient1/crop4000v2/datasets_24_04_22_Cellbound1_4_nuclei_target/cyto3/")

image = imread(path_dapi_folder / image_name)
cyto1 = imread(path_cyto1_folder / image_name)
cyto2 = imread(path_cyto2_folder / image_name)
cyto3 = imread(path_cyto3_folder / image_name)    
    

In [4]:
### vizualise + interactive anntotion with napari
viewer = napari.Viewer()
viewer.add_image(image, name='dapi')
viewer.add_image(cyto1, name='cyto1')
viewer.add_image(cyto2, name='cyto2')
viewer.add_image(cyto3, name='cyto3')

napari.manifest -> 'napari' could not be imported: Could not find file 'builtins.yaml' in module 'napari'


<Image layer 'cyto3' at 0x7ff9d80f5f70>

### use the shape layer for annotation
https://napari.org/stable/howtos/layers/shapes.html

In [5]:
def from_shape_layer_to_mask(viewer,
                             layer_shape_name = "Shapes [1]",
                            image_shape = (400, 400)
                             ):
    final_mask = np.zeros(image_shape, dtype=np.uint16)
    for layer in  viewer.layers.__dict__["_list"]:
        if layer._name==layer_shape_name:
            # Step 1: Retrieve the polygon coordinates
            polygon = layer.data
            # Step 2: Create a meshgrid
            y, x = np.mgrid[:image.shape[0], :image.shape[1]]
            # Step 3: Create a path object from the polygon coordinates
            for index_cell in range(len(polygon)):
                if polygon[index_cell].shape[-1] == 3:
                    polygon[index_cell] = polygon[index_cell][:, 1:]
                    print("3D shape detected, only the two last columns are kept 3D annotation are not supported yet")
                poly_path = mplPath.Path(polygon[index_cell])
                # Step 4: Create a boolean mask
                mask = poly_path.contains_points(np.vstack((x.flatten(), y.flatten())).T).reshape(x.shape)
                # Step 5: Convert the boolean mask to an integer mask
                mask = mask.astype(np.uint16).T
                final_mask[mask>0] = index_cell+1
    return final_mask

### save your annotation

In [9]:
final_mask = from_shape_layer_to_mask(
                    viewer,
                    layer_shape_name = "Shapes",
                    image_shape = image.shape,
                     )
path_to_save_annotation = Path("")
tifffile.imwrite(path_to_save_annotation / ('tom_' + image_name), final_mask.astype(np.uint16))


### double check you annotation 

In [ ]:
viewer.add_image(final_mask, name='final_mask')
